In [2]:
import pandas as pd
from django.core.exceptions import ObjectDoesNotExist

In [3]:
fabric_scheme, _ = SkosConceptScheme.objects.get_or_create(
    dc_title='fabric'
)
painting_style_scheme, _ = SkosConceptScheme.objects.get_or_create(
    dc_title='painting_style'
)
painting_sub_technique_scheme, _ = SkosConceptScheme.objects.get_or_create(
    dc_title='painting_sub_technique'
)

In [4]:
file = "dobjects\data\metadata_ODEEG-archaeology.xlsx"

In [5]:
df = pd.read_excel(file).fillna('False')

In [10]:
for i, row in df.iterrows():
    if row['Folder name'] != 'False':
        try:
            col = Collection.objects.get(has_title__endswith=row['Folder name'])
        except ObjectDoesNotExist:
            col = None
        
        if col:
            temp, _ = DigitalContainer.objects.get_or_create(
                belongs_to=col
            )
        else:
            temp = None
        
        if temp:
            if row['ID Inv.Nr.'] != 'False':
                temp.id_inv_nr = row['ID Inv.Nr.']
            if row['BAPD Nr.'] != 'False':
                temp.bapd_nr = str(row['BAPD Nr.'])
            else: 
                temp.bapd_nr = ""
            temp.folder_name = row['Folder name']
            
            if row['Object is associated to [ID Inv.Nr.]'] != 'False':
                rel, _ = DigitalContainer.objects.get_or_create(
                    id_inv_nr = row['ID Inv.Nr.']
                )
                temp.related_object.add(rel)

            if row['Fabric (as in BAPD)'] != 'False':
                fabric, _ = SkosConcept.objects.get_or_create(
                    pref_label=row['Fabric (as in BAPD)']
                )
                fabric.scheme.set([fabric_scheme])
                temp.fabric.set([fabric])
            if row['Painting style/technique'] != 'False':
                painting_style, _ = SkosConcept.objects.get_or_create(
                    pref_label=row['Painting style/technique']
                )
                fabric.scheme.set([painting_style_scheme])
                temp.painting_style.set([painting_style])
            if row['Painting sub technique'] != 'False':
                painting_sub_technique, _ = SkosConcept.objects.get_or_create(
                    pref_label=row['Painting sub technique']
                )
                fabric.scheme.set([painting_sub_technique_scheme])
                temp.painting_sub_technique.set([painting_sub_technique])
                        
                
            if row['Collection reference [institution] [permalink]'] != 'False':
                temp.collection_reference = row['Collection reference [institution] [permalink]']
            else:
                temp.collection_reference = ""
            
            if row['CVA Reference'] != 'False':
                temp.cva_ref = row['CVA Reference']
            else:
                temp.cva_ref = ""
            
            if row['Weight [g] [AAT ID: 300056240]'] != 'False':
                temp.weight = row['Weight [g] [AAT ID: 300056240]']
            
            if row['Height (max.) [mm] [AAT ID: 300055644]'] != 'False':
                temp.height = row['Height (max.) [mm] [AAT ID: 300055644]']
            
            if row['Width (max.) [mm] [AAT ID: 300055647]'] != 'False':
                temp.width = row['Width (max.) [mm] [AAT ID: 300055647]']
            
            if row['Length (max.) [mm] [AAT ID: 300055645]'] != 'False':
                temp.length = row['Length (max.) [mm] [AAT ID: 300055645]']
            
            if row["Filling Height [mm] ['filling' AAT ID: 300053092; 'height': 300055644]"] != 'False':
                temp.filling_height = row["Filling Height [mm] ['filling' AAT ID: 300053092; 'height': 300055644]"]
            
            if row["Filling Volume [cm3] ['filling' AAT ID: 300053092; 'volume': 300055649]"] != 'False':
                temp.filling_volume = row["Filling Volume [cm3] ['filling' AAT ID: 300053092; 'volume': 300055649]"]
            
            if row["Material Volume [cm3] ['material': AAT ID: 300010358; 'volume': 300055649]"] != 'False':
                temp.material_volume = row["Material Volume [cm3] ['material': AAT ID: 300010358; 'volume': 300055649]"]
            
            if row["Material Density [g/cm3] ['material': AAT ID: 300010358; 'density': 300056237]"] != 'False':
                temp.material_density = row["Material Density [g/cm3] ['material': AAT ID: 300010358; 'density': 300056237]"]
            
            if row["Outer Volume [cm3] ['volume' AAT ID: 300055649]"] != 'False':
                temp.outer_volume = row["Outer Volume [cm3] ['volume' AAT ID: 300055649]"]
            
            if row["Provenance: attributed to artist/maker ['provenance' AAT ID: 300055863]"] != 'False':
                temp.prov_attr_artist = row["Provenance: attributed to artist/maker ['provenance' AAT ID: 300055863]"]
            else:
                temp.prov_attr_artist = ""
            
            if row["Provenance: finding spot [AAT ID 'possible', not all facts/scholars agree: 300404777; 'undetermined': 300379012; 'unavailable': 300400512]"] != 'False':
                temp.pl_find_cert = row["Provenance: finding spot [AAT ID 'possible', not all facts/scholars agree: 300404777; 'undetermined': 300379012; 'unavailable': 300400512]"]
            
            if row['Period (date range) [AAT ID: 300081446]'] != 'False':
                years = row['Period (date range) [AAT ID: 300081446]'].replace(' BCE', '').split('-')
                if len(years) > 1:
                    per, _ = Period.objects.get_or_create(
                        period_start_year=int(years[0])*-1,
                        period_end_year=int(years[1])*-1,
                    )
                elif len(years) == 1:
                    per, _ = Period.objects.get_or_create(
                        period_start_year=int(years[0])*-1,
                        period_end_year=int(years[0])*-1,
                    )
                else:
                    per = None

                if per:
                    if row['Period [Chronontology/PeriodO ID]'] != 'False':
                        per.period_url = row['Period [Chronontology/PeriodO ID]']

                    if row['Period [AAT ID: 300081446]'] != 'False':
                        per.name = row['Period [AAT ID: 300081446]']
                    if row['Period/Phase [abbreviation]'] != 'False':
                        per.period_phase = row['Period/Phase [abbreviation]']
                    per.save()
                    temp.period = per
                
            if row['Collection [institution] [AAT ID: 300025976]'] != 'False':
                inst, _ = Institution.objects.get_or_create(
                    written_name=row['Collection [institution] [AAT ID: 300025976]']
                )
                if row['Collection [GeoNameID]'] != 'False':
                    inst.authority_url = row['Collection [GeoNameID]']
                    inst.save()
                if row['Collection [specific] [AAT ID: 300025976]'] != 'False':
                    sam, _ = Institution.objects.get_or_create(
                        written_name=row['Collection [specific] [AAT ID: 300025976]']
                    )
                    sam.parent_institution = inst
                    sam.save()
                    temp.located_at = sam
            
            if row["Provenance: finding spot ['provenance' AAT ID: 300055863]"] != 'False':
                pl = None
                pl, _ = Place.objects.get_or_create(
                    name=row["Provenance: finding spot ['provenance' AAT ID: 300055863]"]
                )
                if row["Provenance: finding spot [GeoNameID]"] != 'False':
                    pl.geonames_id = row["Provenance: finding spot [GeoNameID]"]
                else:
                    pl.geonames_id = ""

                if row["Provenance: finding spot [GeoName exact location Y/N]"] != 'False':
                    pl.geonames_exact = row["Provenance: finding spot [GeoName exact location Y/N]"]
                pl.save()
                temp.pl_find = pl
            
            
            if row["Provenance: production center/workshop ['provenance' AAT ID: 300055863]"] != 'False':
                pl = None
                pl, _ = Place.objects.get_or_create(
                    name=row["Provenance: production center/workshop ['provenance' AAT ID: 300055863]"]
                )
                if row["Provenance: production center/workshop ['provenance' AAT ID: 300055863]"] != 'False':
                    pl.geonames_id = row["Provenance: production center/workshop ['provenance' AAT ID: 300055863]"]
                else:
                    pl.geonames_id = ""
                pl.save()
                temp.pl_prod_center = pl
                
            
            if row["Provenance: place of latest acquisition [modern times] ['provenance' AAT ID: 300055863]"] != 'False':
                pl = None
                pl, _ = Place.objects.get_or_create(
                    name=row["Provenance: place of latest acquisition [modern times] ['provenance' AAT ID: 300055863]"]
                )
                if row["Provenance: place of latest acquisition [modern times] [GeoNameID]"] != 'False':
                    pl.geonames_id = row["Provenance: place of latest acquisition [modern times] [GeoNameID]"]
                else:
                    pl.geonames_id = ""
                if row["Provenance: place of acquisition (modern times) [GeoNameexact location Y/N]"] != 'False':
                    pl.geonames_exact = row["Provenance: place of acquisition (modern times) [GeoNameexact location Y/N]"]
                else:
                    pl.geonames_exact = ""
                pl.save()
                temp.pl_acq = pl
            
            temp.save()

In [ ]:
# Collection.objects.get(has_title__endswith='KHM-ANSA-IV1')